## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-09-15-47-11 +0000,nypost,20 alleged Mossad spies arrested in Iran amid ...,https://nypost.com/2025/08/09/world-news/20-al...
1,2025-08-09-15-36-00 +0000,wsj,Ukraine and Europe Counter Putin’s Cease-Fire ...,https://www.wsj.com/world/ukraine-and-europe-c...
2,2025-08-09-15-35-45 +0000,nyt,Mexico’s President Says U.S. Forces Are Unwelc...,https://www.nytimes.com/2025/08/08/world/ameri...
3,2025-08-09-15-30-15 +0000,nypost,NYC to shut down last migrant hotel after shel...,https://nypost.com/2025/08/09/us-news/nyc-to-s...
4,2025-08-09-15-29-23 +0000,nypost,Health officials confirm human case of West Ni...,https://nypost.com/2025/08/09/us-news/health-o...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2321/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
197,trump,67
135,gaza,18
41,new,15
445,military,13
12,putin,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
134,2025-08-09-01-00-00 +0000,wsj,President Trump is considering reclassifying m...,https://www.wsj.com/politics/policy/trump-mari...,170
269,2025-08-08-16-13-00 +0000,wsj,President Trump has directed the Pentagon to b...,https://www.wsj.com/politics/national-security...,159
49,2025-08-09-12-09-19 +0000,cbc,Zelenskyy says any peace deal without Ukraine ...,https://www.cbc.ca/news/world/zelenskyy-dismis...,131
69,2025-08-09-09-48-43 +0000,nypost,Zelensky vows Russia will not get any Ukrainia...,https://nypost.com/2025/08/09/us-news/zelensky...,127
99,2025-08-09-07-51-56 +0000,bbc,Mexico rules out Trump's reported military pla...,https://www.bbc.com/news/articles/c07pdel9vyjo...,127


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
134,170,2025-08-09-01-00-00 +0000,wsj,President Trump is considering reclassifying m...,https://www.wsj.com/politics/policy/trump-mari...
269,79,2025-08-08-16-13-00 +0000,wsj,President Trump has directed the Pentagon to b...,https://www.wsj.com/politics/national-security...
218,66,2025-08-08-20-03-13 +0000,nypost,Inside Israel’s plan to take over Gaza City an...,https://nypost.com/2025/08/08/world-news/insid...
49,64,2025-08-09-12-09-19 +0000,cbc,Zelenskyy says any peace deal without Ukraine ...,https://www.cbc.ca/news/world/zelenskyy-dismis...
25,46,2025-08-09-13-56-20 +0000,bbc,Atlanta police officer dies after shooting nea...,https://www.bbc.com/news/articles/cp3ekez0k2xo...
260,41,2025-08-08-16-47-10 +0000,nypost,JD Vance goes fishing with top UK official Dav...,https://nypost.com/2025/08/08/us-news/jd-vance...
159,40,2025-08-08-23-04-25 +0000,cbc,Trump says he'll meet Putin in Alaska next Fri...,https://www.cbc.ca/news/world/trump-putin-meet...
147,34,2025-08-08-23-49-00 +0000,wsj,President Trump removed former congressman Bil...,https://www.wsj.com/politics/policy/billy-long...
32,32,2025-08-09-12-59-45 +0000,nypost,Cuomo bashes ‘rich person’ Zohran Mamdani for ...,https://nypost.com/2025/08/09/us-news/cuomo-ba...
10,32,2025-08-09-15-03-32 +0000,nypost,Drunk illegal immigrant driver was 3 times ove...,https://nypost.com/2025/08/09/us-news/new-char...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
